# Tao bang lien ket giua course-exercise/video
Dựa trên tỉ lệ score trung bình học sinh đặt được trên từng course_id

In [1]:
import pandas as pd
import dask.dataframe as dd
import re
import numpy as np

In [2]:
import json

In [3]:
import pandas as pd

# Đọc tệp JSON Lines
course_df = pd.read_json('/kaggle/input/mooccubex-dataset/course.json', lines=True)

# Đọc tệp CSV chứa thông tin top 1000 khóa học
courses1000_df = pd.read_csv('/kaggle/input/1000course/top_1000_courses.csv')

# Đổi tên cột 'course_order' thành 'course_id'
courses1000_df.rename(columns={'course_order': 'course_id'}, inplace=True)

# Thêm tiền tố "C_" vào cột 'course_id'
courses1000_df['course_id'] = courses1000_df['course_id'].apply(lambda x: f"C_{x}")

# Lấy danh sách hợp lệ của course_id
valid_courses = courses1000_df['course_id'].tolist()

# Lọc course_df để chỉ giữ lại các dòng có id thuộc danh sách hợp lệ
course_df_filtered = course_df[course_df['id'].isin(valid_courses)]

print("Filtered course_df:")
course_df_filtered

Filtered course_df:


,id,name,field,prerequisites,about,resource
30,C_674903,不朽的艺术：走进大师与经典,[艺术学],无,【国家精品课】以历史为经线，中外比较为纬线，以文化解读为主导，引导修课学生欣赏和理解中西艺术...,"[{'titles': ['第一讲：艺术概说：问题与方法', ' 1.图像与信息：事实缺损'..."
31,C_674910,图案审美与创作,[],,【国家精品课】本课程提供图案感受美、提炼美、表现美的过程体验。学习图案可以了解历史，感受时代...,"[{'titles': ['第一讲 图案导学1', '1-1图案与时尚、历史、文化等', '..."
32,C_674912,音乐与健康,[],,本课程精心设计了音乐与健康、治疗、养生、音乐处方单等八个专题的内容，适合喜爱音乐和关注身心健...,"[{'titles': ['1 音乐与健康', '1.1 导论', '导论'], 're..."
33,C_674914,管理会计学,[],,本课程顺应新经济发展趋势，主要讲述了管理会计基础理论和实际运用方法，旨在为学员搭建起管理会计...,"[{'titles': ['第一章 总论', '1.1管理会计的基本概念和内容', '1.1..."
35,C_674920,逻辑学概论,[哲学],,“逻辑”是什么？大家都很熟悉却又似乎难以说清。本课程将告诉你：作为一门学科，逻辑学研究的是什...,"[{'titles': ['第一讲 什么是逻辑学', '1.1 “逻辑""和逻辑学', '默认..."
...,...,...,...,...,...,...
3566,C_697821,操作系统,[计算机科学与技术],,疾风计划2021【首期限额招募】进行中\n名校名师丨专属助教丨专业认证丨系统全面丨修炼内功丨...,"[{'titles': ['第零讲 在线教学环境准备', '0.1 Piazza讨论区', ..."
3567,C_697828,程序设计基础,[],,疾风计划2021【首期限额招募】进行中\n名校名师丨专属助教丨专业认证丨系统全面丨修炼内功丨...,"[{'titles': ['第一章 编程初步', '1.1 基础知识', '1.1.1 什..."
3578,C_2297236,物联网概论,[],,在万物互联的时代即将到来之时，让我们一起通过这门课程，认识物联网的基本概念，掌握物联网的关键...,"[{'titles': ['概述', '物联网概述', '起源与发展'], 'resourc..."
3603,C_2308104,武术,[],无,中华武术，博大精深，传承武术精粹，弘扬武术文化，以武修身。,"[{'titles': ['专题一：《武术概述》', '1.1武术的缘起', '武术的缘起'..."


In [4]:
print(course_df_filtered['resource'])

30      [{'titles': ['第一讲：艺术概说：问题与方法', ' 1.图像与信息：事实缺损'...
31      [{'titles': ['第一讲 图案导学1', '1-1图案与时尚、历史、文化等', '...
32      [{'titles': ['1  音乐与健康', '1.1  导论', '导论'], 're...
33      [{'titles': ['第一章 总论', '1.1管理会计的基本概念和内容', '1.1...
35      [{'titles': ['第一讲 什么是逻辑学', '1.1 “逻辑"和逻辑学', '默认...
                              ...                        
3566    [{'titles': ['第零讲 在线教学环境准备', '0.1 Piazza讨论区', ...
3567    [{'titles': ['第一章 编程初步', '1.1 基础知识', '1.1.1  什...
3578    [{'titles': ['概述', '物联网概述', '起源与发展'], 'resourc...
3603    [{'titles': ['专题一：《武术概述》', '1.1武术的缘起', '武术的缘起'...
3647    [{'titles': ['第一章 翻译概述', '1.1翻译的目的', '1.1翻译的目的...
Name: resource, Length: 989, dtype: object


In [5]:
import pandas as pd

# Hàm để tách các resource_id dựa trên điều kiện
def extract_resources(course_id, resources, prefix):
    return [
        {"id": course_id, "resource_id": res['resource_id']}
        for res in resources if res['resource_id'].startswith(prefix)
    ]

# Tạo dataframe chứa video_id
video_data = []
exercise_data = []

for _, row in course_df_filtered.iterrows():
    video_data.extend(extract_resources(row['id'], row['resource'], "V_"))
    exercise_data.extend(extract_resources(row['id'], row['resource'], "Ex_"))

video_df = pd.DataFrame(video_data)
exercise_df = pd.DataFrame(exercise_data)

In [6]:
# Kết quả
print("Course - Video DataFrame:")
# Đổi tên cột 'course_order' thành 'course_id'
video_df.rename(columns={'id': 'course_id'}, inplace=True)
video_df.to_csv("/kaggle/working/course_video.csv")
video_df

Course - Video DataFrame:


,course_id,resource_id
0,C_674903,V_35117
1,C_674903,V_35118
2,C_674903,V_35119
3,C_674903,V_35120
4,C_674903,V_35121
...,...,...
65914,C_893812,V_1058509
65915,C_893812,V_1058510
65916,C_893812,V_1058511
65917,C_893812,V_1058512


In [7]:
print("\nCourse - Exercise DataFrame:")
exercise_df.rename(columns={'id': 'course_id'}, inplace=True)
exercise_df.to_csv("/kaggle/working/course_exercise.csv")
exercise_df


Course - Exercise DataFrame:


,course_id,resource_id
0,C_674903,Ex_35127
1,C_674903,Ex_35136
2,C_674903,Ex_35145
3,C_674903,Ex_35153
4,C_674903,Ex_35162
...,...,...
23016,C_893812,Ex_1058493
23017,C_893812,Ex_1058497
23018,C_893812,Ex_1058507
23019,C_893812,Ex_1058513


In [8]:
# Đọc dữ liệu từ tệp user-problem.json
user_problem_df = pd.read_json('/kaggle/input/mooccubex-dataset/user-problem.json', lines=True)

# Lọc bảng user_problem_df để chỉ lấy các dòng có user_id trong user_df_filtered
valid_user_ids = user_df_filtered['id'].tolist()
user_problem_df_filtered = user_problem_df[user_problem_df['user_id'].isin(valid_user_ids)]

# In kết quả lọc từ user_problem_df
print("\nFiltered user_problem_df:")
user_problem_df_filtered

In [ ]:
import pandas as pd

# Đọc dữ liệu từ tệp exercise-problem.txt
exercise_problem_df = pd.read_csv('/kaggle/input/mooccubex-dataset/exercise-problem.txt', sep="\t", header=None, names=["exercise_id", "problem_id"])

# Lọc bảng exercise_problem_df để chỉ lấy các dòng có problem_id trong user_problem_df_filtered
valid_problem_ids = user_problem_df_filtered['problem_id'].tolist()
exercise_problem_df_filtered = exercise_problem_df[exercise_problem_df['problem_id'].isin(valid_problem_ids)]

# In kết quả lọc từ exercise_problem_df
print("\nFiltered exercise_problem_df:")
exercise_problem_df_filtered

In [ ]:
pip install tqdm

In [ ]:
import pandas as pd
import json
from datetime import datetime
from tqdm import tqdm  # Thêm thư viện tqdm

# Chuyển đổi thời gian từ chuỗi sang đối tượng datetime
def convert_to_datetime(time_str):
    return datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")

# Lấy thông tin khóa học và các bài tập tương ứng từ dữ liệu
def get_submission_course_mapping(user_data, user_problem_data, exercise_problem_data):
    # Lấy thời gian nộp bài
    submit_time = convert_to_datetime(user_problem_data["submit_time"])
    
    # Danh sách lưu trữ kết quả
    result_data = []

    # Lọc các problem_id từ bảng user_problem_data và bảng exercise_problem_data
    filtered_exercise_problems = exercise_problem_data[exercise_problem_data['problem_id'] == user_problem_data["problem_id"]]

    # Nếu có các bài tập hợp lệ, tiếp tục xử lý
    if not filtered_exercise_problems.empty:
        # Duyệt qua từng bài tập trong exercise_problem_data
        for _, row in filtered_exercise_problems.iterrows():
            exercise_id = row['exercise_id']
            problem_id = row['problem_id']

            # Duyệt qua từng khóa học trong course_order và kiểm tra thời gian enroll
            for j, course_id in enumerate(user_data["course_order"]):
                enroll_start_time = convert_to_datetime(user_data["enroll_time"][j])
                enroll_end_time = convert_to_datetime(user_data["enroll_time"][j + 1] if j + 1 < len(user_data["enroll_time"]) else user_data["enroll_time"][j])

                # Kiểm tra xem thời gian nộp bài có nằm trong khoảng thời gian học hay không
                if enroll_start_time <= submit_time <= enroll_end_time:
                    # Thêm exercise_id vào kết quả cho khóa học tương ứng
                    if course_id not in [result['course_id'] for result in result_data]:
                        result_data.append({"course_id": course_id, "exercise_ids": [exercise_id]})
                    else:
                        # Nếu khóa học đã có trong kết quả, thêm exercise_id vào chuỗi
                        for result in result_data:
                            if result["course_id"] == course_id:
                                result["exercise_ids"].append(exercise_id)
                                break

    # Chuyển danh sách kết quả thành DataFrame, với exercise_ids là chuỗi bài tập
    result_df = pd.DataFrame(result_data)
    # Chuyển exercise_ids thành chuỗi các bài tập
    result_df['exercise_ids'] = result_df['exercise_ids'].apply(lambda x: ', '.join(x))
    return result_df

# Duyệt qua các bản ghi trong bảng user_problem_df_filtered và lấy kết quả cho từng người dùng
final_results = []

# Thêm thanh tiến trình vào vòng lặp duyệt qua các bản ghi user_problem_df_filtered
for _, user_problem_row in tqdm(user_problem_df_filtered.iterrows(), total=user_problem_df_filtered.shape[0], desc="Processing user problems"):
    user_id = user_problem_row["user_id"]
    # Lấy thông tin người dùng từ user_df_filtered
    user_data = user_df_filtered[user_df_filtered["id"] == user_id].iloc[0]  # Lấy thông tin người dùng
    result_df = get_submission_course_mapping(user_data, user_problem_row, exercise_problem_df_filtered)
    
    # Gộp kết quả cho người dùng
    final_results.append(result_df)

# Kết quả cuối cùng là DataFrame với course_id và exercise_ids cho từng người dùng
course_exercise = pd.concat(final_results, ignore_index=True)

# In kết quả
course_exercise

In [ ]:
course_exercise.to_csv("/kaggle/working/course_exercise.csv")

In [ ]:
user_problem_score_df = cudf.read_csv('/content/drive/MyDrive/CS313/user_problem_score_df.csv')
user_problem_score_df

In [ ]:
x = user_problem_score_df[user_problem_score_df['problem_id'] == '47064']
print(x)

In [ ]:
user_problem_score_df[user_problem_score_df['score'] == -1]

In [ ]:
problem_df = cudf.read_csv('/content/drive/MyDrive/CS313/problem_df.csv')
problem_df

In [ ]:
count_zeros = (problem_df['score'] == 0).sum()
print(count_zeros)

In [ ]:
problem_df[problem_df['problem_id'] == 47064]

## Hàm tính problem_score

In [ ]:
# Hàm tính tỷ lệ
def calculate_average_ratio(problem_id):
    # Truy xuất score từ problem_df
    problem_score = problem_df[problem_df['problem_id'] == problem_id]['score'].values
    # Truy xuất user_score từ user_problem_score_df
    user_score = user_problem_score_df[user_problem_score_df['problem_id'] == problem_id]['problem_score'].values

    # Kiểm tra xem có tồn tại score và user_score hay không
    if len(problem_score) > 0 and len(user_score) > 0:
        score_chuan = problem_score[0]
        user_score = user_score[0]
        if user_score > 0 and score_chuan > 0:
            ratio = user_score / score_chuan
        else:
            ratio = 0
    else:
        ratio = 0

    return cudf.DataFrame({'problem_id': [problem_id], 'score': [ratio]})

In [ ]:
x = calculate_average_ratio(47064)
x

In [ ]:
problem_ids = problem_df['problem_id'].to_arrow().to_pylist()
results = [calculate_average_ratio(id) for id in problem_ids]
result_df = cudf.concat(results)

In [ ]:
problem_ids = problem_df['problem_id'].to_arrow().to_pylist()
len(problem_ids)

In [ ]:
count_zeros = (result_df['score'] == 0).sum()
print(count_zeros)

In [ ]:
file_path = '/content/drive/MyDrive/CS313/problem_score.csv'
result_df.to_csv(file_path, index=False)

print(f'DataFrame đã được lưu vào {file_path}')

## Exercise_score

In [ ]:
exercise_problem_df = cudf.read_csv('/content/drive/MyDrive/CS313/exercise_problemlist.csv')
exercise_problem_df

In [ ]:
exercise_problem_df.info()

In [ ]:
problem_score_df = cudf.read_csv('/content/drive/MyDrive/CS313/problem_score.csv')
problem_score_df

In [ ]:
# Function to calculate the average score for each exercise_id
def calculate_average_score(problem_list):
    # Split the problem_list into individual IDs
    problem_ids_str = problem_list.split(',')
    problem_ids = []

    # Convert to integers, ignoring non-integer values
    for pid in problem_ids_str:
        try:
            problem_ids.append(int(pid))
        except ValueError:
            continue

    # Retrieve scores from problem_score_df
    scores = []
    for problem_id in problem_ids:
        score = problem_score_df[problem_score_df['problem_id'] == problem_id]['score']
        if len(score) > 0:
            scores.append(score.iloc[0])
        else:
            scores.append(0)

    # Calculate the average score
    average_score = np.mean(scores) if scores else 0
    return float(average_score)

In [ ]:
def ex_score(ex_id):
    # Find the corresponding index for ex_id in exercise_problem_df
    index = exercise_problem_df[exercise_problem_df['exercise_id'] == ex_id].index[0]

    # Get the value of problem_ids at the corresponding index
    problem_ids = exercise_problem_df['problem_ids'].iloc[index]
    score = calculate_average_score(problem_ids)
    return cudf.DataFrame({'exercise_id': [ex_id], 'score': [score]})

In [ ]:
x = ex_score(8825283)
x

In [ ]:
ex_ids = exercise_problem_df['exercise_id'].to_arrow().to_pylist()
results = [ex_score(id) for id in ex_ids]
result_df = cudf.concat(results)

In [ ]:
count_zeros = (result_df['score'] == 0).sum()
print(count_zeros)

In [ ]:
result_df

In [ ]:
result_df.to_csv("/kaggle/working/exercise_score.csv", index=False)

##Course_score

In [ ]:
course_exercise_df = pd.read_csv('/content/drive/MyDrive/CS313/course-exercise.txt',header=None)
course_exercise_df

In [ ]:
course_exercise_df[0]

In [ ]:
# Tách cột 'combined' thành hai cột 'exercise_id' và 'problem_id'
course_exercise_df[['course_id', 'exercise_id']] = course_exercise_df[0].str.split('\\t', expand=True)

# Xóa cột 'combined' vì không còn cần thiết nữa
course_exercise_df.drop(columns=[0], inplace=True)

# Chuyển đổi các cột thành kiểu dữ liệu phù hợp (nếu cần thiết)
course_exercise_df['course_id'] = course_exercise_df['course_id'].astype(int)
course_exercise_df['exercise_id'] = course_exercise_df['exercise_id'].astype(int)

# Nhóm các problem_id theo exercise_id
grouped = course_exercise_df.groupby('course_id')['exercise_id'].apply(list).reset_index()

# Đổi tên cột kết quả cho rõ ràng hơn
grouped.columns = ['course_id', 'exercise_ids']

# Hiển thị kết quả
print(grouped)

In [ ]:
result_df = pd.DataFrame(grouped)
file_path = '/content/drive/MyDrive/CS313/course_exlist.csv'
result_df.to_csv(file_path, index=False)

print(f'DataFrame đã được lưu vào {file_path}')

In [ ]:
course_exerciselist_df = cudf.read_csv('/content/drive/MyDrive/CS313/course_exlist.csv')
course_exerciselist_df

In [ ]:
ex_score_df = cudf.read_csv('/content/drive/MyDrive/CS313/exercise_score.csv')
ex_score_df

In [ ]:
# Function to calculate the average score for each exercise_id
def calculate_average_score(ex_list):
    # Split the problem_list into individual IDs
    ex_ids_str = ex_list.split(',')
    ex_ids = []

    # Convert to integers, ignoring non-integer values
    for eid in ex_ids_str:
        try:
            ex_ids.append(int(eid))
        except ValueError:
            continue

    # Retrieve scores from problem_score_df
    scores = []
    for ex_id in ex_ids:
        score = ex_score_df[ex_score_df['exercise_id'] == ex_id]['score']
        if len(score) > 0:
            scores.append(score.iloc[0])
        else:
            scores.append(0)

    # Calculate the average score
    average_score = np.mean(scores) if scores else 0
    return float(average_score)

In [ ]:
def course_score(course_id):
    # Find the corresponding index for ex_id in exercise_problem_df
    index = course_exerciselist_df[course_exerciselist_df['course_id'] == course_id].index[0]

    # Get the value of problem_ids at the corresponding index
    ex_ids = course_exerciselist_df['exercise_ids'].iloc[index]
    score = calculate_average_score(ex_ids)
    return cudf.DataFrame({'course_id': [course_id], 'score': [score]})

In [ ]:
x = course_score(597225)
x

In [ ]:
course_ids = course_exerciselist_df['course_id'].to_arrow().to_pylist()
results = [course_score(id) for id in course_ids]
result_df = cudf.concat(results)

In [ ]:
count_zeros = (result_df['score'] == 0).sum()
print(count_zeros)

In [ ]:
file_path = '/content/drive/MyDrive/CS313/course_score.csv'
result_df.to_csv(file_path, index=False)

print(f'DataFrame đã được lưu vào {file_path}')

##Full Course_score

In [ ]:
course_df = dd.read_json('/content/drive/MyDrive/CS313/course.json')
course_df

In [ ]:
course_df = course_df.iloc[:, [0]]

# Compute and print the resulting DataFrame to verify
print(course_df.compute())

In [ ]:
file_path = '/content/drive/MyDrive/CS313/course_id.csv'
course_df.to_csv(file_path, index=False)

print(f'DataFrame đã được lưu vào {file_path}')

In [ ]:
course_id_df = cudf.read_csv('/content/drive/MyDrive/CS313/course_id.csv/course_id.txt')
course_id_df

In [ ]:
course_score_df = cudf.read_csv('/content/drive/MyDrive/CS313/course_score.csv')
course_score_df

In [ ]:
# Initialize an empty list to store the results
results = []

# Iterate through each course_id in course_id_df
for course_id in course_id_df['id'].to_arrow().to_pylist():
    stripped_course_id = int(course_id.split('_')[1])

    # Try to get the score for the current stripped_course_id
    score = course_score_df[course_score_df['course_id'] == stripped_course_id]['score']

    # If the score exists, get the value; otherwise, assign 0
    if len(score) > 0:
        results.append((course_id, score.iloc[0]))
    else:
        results.append((course_id, 0))

# Create a new DataFrame with the results
result_df = cudf.DataFrame(results, columns=['course_id', 'score'])

# Display the result DataFrame
print(result_df)

In [ ]:
count_zeros = (result_df['score'] == 0).sum()
print(count_zeros)

In [ ]:
file_path = '/content/drive/MyDrive/CS313/full_course_score.csv'
result_df.to_csv(file_path, index=False)

print(f'DataFrame đã được lưu vào {file_path}')

In [ ]:
full_course_score_df = cudf.read_csv('/content/drive/MyDrive/CS313/full_course_score.csv')
full_course_score_df

## Gán nhanx Xếp hạng 1 - 5

In [ ]:
full_course_score_df = full_course_score_df.to_pandas()

In [ ]:
# Calculate the quantiles for the scores
quantiles = full_course_score_df['score'].quantile([0.2, 0.8, 0.85, 0.95])

# Function to assign rank based on score
def assign_rank(score, quantiles):
    if score <= quantiles.loc[0.2]:
        return 1
    elif score <= quantiles.loc[0.8]:
        return 2
    elif score <= quantiles.loc[0.85]:
        return 3
    elif score <= quantiles.loc[0.95]:
        return 4
    else:
        return 5

# Apply the ranking function to the score column
full_course_score_df['rank'] = full_course_score_df['score'].apply(lambda x: assign_rank(x, quantiles))

# Display the result DataFrame
print(full_course_score_df)

In [ ]:
file_path = '/content/drive/MyDrive/CS313/course_score_rank.csv'
full_course_score_df.to_csv(file_path, index=False)

print(f'DataFrame đã được lưu vào {file_path}')

In [ ]:
import matplotlib.pyplot as plt
# Plot the distribution of course_ids for each rank
rank_counts = full_course_score_df['rank'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
rank_counts.plot(kind='bar')
plt.xlabel('Rank')
plt.ylabel('Number of Courses')
plt.title('Distribution of Courses by Rank')
plt.xticks(rotation=0)
plt.show()

##Course

In [ ]:
course_df = dd.read_json('/content/drive/MyDrive/CS313/data/entities/course.json')
course_df.head()

In [ ]:
x = course_df.loc[course_df['id'] == 'C_584313']['resource'].compute()
x

In [ ]:
ex_problem_df = pd.read_csv('/content/drive/MyDrive/CS313/data/Input/exercise-problem.txt',header=None)
ex_problem_df

In [ ]:
def ex_pro(ex_id): #ex_id => pro_id
  ex_id = ex_id.split('_')[1]
  ex_list = []
  result_list = []

    # Duyệt qua từng phần tử trong cột văn bản của DataFrame
  for text in ex_problem_df[0]:
    text = text.split('\t')
    if text[0] == ex_id:
      result_list.append(text[1])
  return result_list

In [ ]:
def process_course_id(course_id):
  resource = course_df.loc[course_df['id'] == course_id]['resource'].compute()
  if len(resource) == 0:
    return cudf.DataFrame({'course_id': [course_id], 'score': [None]})
  pattern = r'Ex_\w*'
  ex_id = re.findall(pattern, resource.iloc[0]) #Ex_12344
  problem_list = []
  for i in ex_id:
    problem_list += ex_pro(i)
  score = calculate_average_ratio(problem_list)
  return cudf.DataFrame({'course_id': [course_id], 'score': [score]})

In [ ]:
x = process_course_id('C_584313')
x

In [ ]:
# Lấy giá trị course_id từ course_df
course_ids = course_df['id'].compute()

# Áp dụng hàm process_course_id cho từng course_id và tạo một danh sách các DataFrame kết quả
results = [process_course_id(course_id) for course_id in course_ids]

# Kết hợp các DataFrame trong danh sách lại thành một DataFrame duy nhất
result_df = cudf.concat(results)

In [ ]:
result_df = cudf.concat(results)

In [ ]:
file_path = '/content/drive/MyDrive/CS313/course_score.csv'
result_df.to_csv(file_path, index=False)

print(f'DataFrame đã được lưu vào {file_path}')